# Traitement des données
Pour plus d'information sur les sources de données, lisez le README.md

In [1]:
import pandas as pd
import numpy as np

## 1. Traitement de RP (exploitation complémentaire) pour calculer les effectifs de CSP par commune

In [ ]:
# Télécharger et importer le fichier population et csp (à télécharger ici : https://catalogue-donnees.insee.fr/fr/catalogue/recherche/DS_RP_POPULATION_COMP "Télécharger la totalité du jeu de données")
df = pd.read_csv(r'DS_RP_POPULATION_COMP_2022_data.csv', sep=';')

/tmp/ipykernel_47872/4169547382.py:2: DtypeWarning: Columns (1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'DS_RP_POPULATION_COMP_2022_data.csv', sep=';')


In [ ]:
# Sélection des communes uniquement, de l'année 2022, du sexe _T et de la tranche d'âge 'Y_GE15'
df1 = df[df['GEO_OBJECT'] == 'COM']
df1 = df1[df1['TIME_PERIOD'] == 2022]
df1 = df1[df1['SEX'] == '_T']
df1 = df1[df1['AGE'] == 'Y_GE15']

In [ ]:
# Drop des colonnes inutiles
df1 = df1.drop(columns=['GEO_OBJECT', 'TIME_PERIOD', 'SEX', 'RP_MEASURE', 'AGE'])

In [ ]:
# Arrondis les valeurs de OBS à l'entier supérieur
df1["OBS_VALUE"] = np.ceil(df1["OBS_VALUE"])

In [ ]:
df1.head()

,GEO,PCS,OBS_VALUE
731762,68157,6,160
731794,68340,3,16
731797,68329,6,144
731810,68167,3,21
731816,69141,3,694


In [ ]:
# Exporter df1 en CSV
df1.to_csv('population_communes_2022_csp.csv', index=False)

## 2. Traitement des données de l'INSEE sur la population municipale, le nombre de conseillers municipaux, la proportion de locataires et le taux de pauvreté

### 2.A. Population municipale et nombre de conseillers municipaux

In [21]:
# Importer le fichier population municipale totale (population de référence) (à télécharger ici : https://catalogue-donnees.insee.fr/fr/catalogue/recherche/DS_POPULATIONS_REFERENCE "Télécharger la totalité du jeu de données")
df2 = pd.read_csv(r'data/raw_data/DS_POPULATIONS_REFERENCE_data.csv', sep=';')

# Sélectionner les communes, l'année 2022 et la population municipale (PMUN)
df2 = df2[df2['TIME_PERIOD'] == 2022]
df2 = df2[df2['GEO_OBJECT'] == 'COM']
df2 = df2[df2['POPREF_MEASURE'] == 'PMUN']

# Supprimer les colonnes GEO_OBJECT, TIME_PERIOD, POPREF_MEASURE et FREQ
df2 = df2.drop(columns=['GEO_OBJECT', 'TIME_PERIOD', 'POPREF_MEASURE', 'FREQ'])

# Renommer la colonne OBS_VALUE en POPULATION_MUNICIPALE
df2 = df2.rename(columns={'OBS_VALUE': 'POPULATION_MUNICIPALE'})

In [18]:

def calculer_conseillers_municipaux(fichier_repartition):
   """
   Calcule le nombre de conseillers municipaux pour chaque commune à partir du fichier de population municipale (df2) et du fichier "conseillers.csv" qui fournit la répartition selon les fourchettes de population.
   
   Parameter:
   -----------
   fichier_repartition : str
       Chemin vers le fichier CSV avec la répartition population/conseillers
       
   Returns:
   --------
   pandas.DataFrame
       DataFrame avec les communes et leur nombre de conseillers
   """
   
   # Charger le fichier
   repartition_df = pd.read_csv(fichier_repartition)
   
   # Charger le dataframe population
   communes_df = df2
   
   # Fonction pour déterminer le nombre de conseillers
   def get_nb_conseillers(population):
       for _, row in repartition_df.iterrows():
           if row['population_min'] <= population <= row['population_max']:
               return row['nombre_de_conseillers']
       return None
   
   # Calculer le nombre de conseillers pour chaque commune
   communes_df['NB_CONSEILLERS_TOTAL'] = communes_df['POPULATION_MUNICIPALE'].apply(get_nb_conseillers)
   
   return communes_df

In [22]:
# Lancer la fonction avec le fichier de répartition
df2 = calculer_conseillers_municipaux('./data/conseillers.csv')

In [23]:
df2.head()

,GEO,POPULATION_MUNICIPALE,NB_CONSEILLERS_TOTAL
4858,03066,321,11
4872,07049,60,7
4874,04101,139,11
4875,04067,59,7
4889,12268,424,11


In [ ]:
# Exporter df2 en CSV
# df2.to_csv('population_municipale_2022_et_conseillers.csv', index=False)

### 2.B. Traitement de RP (logement) pour calculer la proportion de locataires par commune

In [57]:
df3 = pd.read_csv("data/raw_data/DS_RP_LOGEMENT_PRINC_2022_data.csv", sep=";")

# Sélectionner les communes, l'année 2022, la mesure RP_MEASURE = 'DWELLINGS_POPSIZE', et L_STAY = '_T'
df3 = df3[df3['TIME_PERIOD'] == 2022]
df3 = df3[df3['GEO_OBJECT'] == 'COM']
df3 = df3[df3['RP_MEASURE'] == 'DWELLINGS_POPSIZE']
df3 = df3[df3['L_STAY'] == '_T']

# Supprimer les colonnes GEO_OBJECT, TIME_PERIOD, OCS, L_STAY, TDW, NRG_SRC, CARS, RP_MEASURE, CARPARK, NOR and BUILD_END
df3 = df3.drop(columns=['GEO_OBJECT', 'TIME_PERIOD', 'OCS', 'L_STAY', 'TDW', 'NRG_SRC', 'CARS', 'RP_MEASURE', 'CARPARK', 'NOR', 'BUILD_END'])

# Fusionner df2 et df3 sur la colonne 'GEO'
df3 = pd.merge(df3, df2[['GEO', 'POPULATION_MUNICIPALE', 'NB_CONSEILLERS_TOTAL']], on='GEO', how='left')

/tmp/ipykernel_7161/1108358151.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df3 = pd.read_csv("data/raw_data/DS_RP_LOGEMENT_PRINC_2022_data.csv", sep=";")


In [ ]:
df3



In [ ]:
# Pivoter le dataframe pour avoir une colonne par TSH
df3_pivot = df3.pivot_table(index='GEO', columns='TSH', values='OBS_VALUE', aggfunc='first').reset_index()

# REGARDER POURQUOI LE PIVOT SUPPRIMER LA COLONNE POPULATION_MUNICIPALE ET NB_CONSEILLERS_TOTAL, ENSUITE REGARDER SI DES TOTAUX SONT MANQUANTS (le merged df2 et df3 les a peut-être supprimées)

# # Catégories de locataires
# categories_locataires = ['211', '212_222', '221', '300']

# # Calculer le total des locataires
# df3_pivot['TOTAL_LOCATAIRES'] = df3_pivot[categories_locataires].sum(axis=1)

# # Si _T est manquant (NaN), remplacer la valeur par la valeur POPULATION_MUNICIPALE
# df3_pivot['TOTAL_HABITANTS'] = df3_pivot['_T'].fillna(df3_pivot['POPULATION_MUNICIPALE'])


# # Calculer la proportion de locataires
# df3_pivot['PROPORTION_LOCATAIRES'] = df3_pivot.apply(
#     lambda row: (row['TOTAL_LOCATAIRES'] / row['TOTAL_HABITANTS'] * 100) if row['TOTAL_HABITANTS'] > 0 else 0,
#     axis=1
# )



df3_pivot.head()

TSH,GEO,100,211,212_222,221,300,_T
0,1077,NaN,NaN,3.0,NaN,NaN,NaN
1,2017,NaN,NaN,3.0,NaN,NaN,NaN
2,2072,NaN,NaN,1.0,NaN,NaN,NaN
3,3149,NaN,NaN,1.0,NaN,NaN,NaN
4,3164,NaN,NaN,NaN,4.0,NaN,NaN


In [34]:
df3.head()

,GEO,TSH,OBS_VALUE
3985791,68374,300,146.00000
3985792,02370,211,106.00000
3985833,39477,212_222,1.03992
3985847,34149,212_222,3.21222
3985853,38326,212_222,1.02414
